# Stage 4d: Defensive Features Ablation

**Question**: Do defensive features (xGC, clean sheets, saves) improve decision quality?

**Hypothesis**: Defenders and goalkeepers earn points through clean sheets and saves. Adding defensive signals may improve predictions for these positions.

**Features to test**:
- `xgc_per90`: Expected goals conceded per 90 (lower = better defense)
- `clean_sheet_rate`: Clean sheet rate over last 5 GWs
- `saves_per90`: Saves per 90 (GKP only, but useful)

**Evaluation**: Compare captain/transfer regret with vs without these features.

In [1]:
import sys
from pathlib import Path

# Add src to path
project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root / "src"))

import pandas as pd
import numpy as np
from copy import deepcopy

from dugout.production.data.reader import DataReader
from dugout.production.features.builder import FeatureBuilder
from dugout.production.features.definitions import FEATURE_COLUMNS, BASE_FEATURES

## 1. Load Raw Data

In [2]:
reader = DataReader()
raw_df = reader.get_all_gw_data()
print(f"Loaded {len(raw_df):,} rows, {raw_df['player_id'].nunique()} players")
print(f"GW range: {raw_df['gw'].min()}-{raw_df['gw'].max()}")
print(f"\nColumns available: {list(raw_df.columns)}")

Loaded 17,362 rows, 803 players
GW range: 1-23

Columns available: ['player_id', 'player_name', 'first_name', 'second_name', 'team_name', 'team_id', 'position', 'now_cost', 'status', 'gw', 'total_points', 'minutes', 'goals_scored', 'assists', 'clean_sheets', 'goals_conceded', 'bonus', 'bps', 'ict_index', 'influence', 'creativity', 'threat', 'xG', 'xA', 'xGI', 'xGC', 'starts', 'is_home', 'opponent_id', 'opponent_name', 'opponent_short', 'opponent_strength', 'fixture_home_goals', 'fixture_away_goals', 'fixture_finished']


## 2. Check Defensive Columns Exist

In [3]:
# Check for defensive columns
defensive_cols = ['expected_goals_conceded', 'xGC', 'clean_sheets', 'saves', 'goals_conceded']
for col in defensive_cols:
    if col in raw_df.columns:
        non_null = raw_df[col].notna().sum()
        print(f"✓ {col}: {non_null:,} non-null values, mean={raw_df[col].mean():.3f}")
    else:
        print(f"✗ {col}: NOT FOUND")

✗ expected_goals_conceded: NOT FOUND
✓ xGC: 17,362 non-null values, mean=0.398
✓ clean_sheets: 17,362 non-null values, mean=0.081
✗ saves: NOT FOUND
✓ goals_conceded: 17,362 non-null values, mean=0.403


## 3. Define Defensive Feature Builder (Research Only)

In [4]:
class DefensiveFeatureBuilder(FeatureBuilder):
    """Extended feature builder with defensive metrics.
    
    RESEARCH ONLY - not for production use.
    """
    
    def _compute_defensive_per90(self, last5: pd.DataFrame) -> dict:
        """Compute defensive per90 features.
        
        Features:
            - xgc_per90: Expected goals conceded per 90 (lower = better defense)
            - clean_sheet_rate: Proportion of games with clean sheet
            - saves_per90: Saves per 90 minutes
        """
        total_mins = last5['minutes'].sum() if 'minutes' in last5.columns else 0
        games_played = (last5['minutes'] > 0).sum() if 'minutes' in last5.columns else 0
        
        # Handle xGC column variants
        xgc_col = 'xGC' if 'xGC' in last5.columns else 'expected_goals_conceded'
        xgc_total = last5[xgc_col].sum() if xgc_col in last5.columns else 0
        
        def to_per90(total: float) -> float:
            return (total / total_mins * 90) if total_mins > 0 else 0.0
        
        # Clean sheet rate (proportion of games with CS)
        cs_total = last5['clean_sheets'].sum() if 'clean_sheets' in last5.columns else 0
        cs_rate = cs_total / games_played if games_played > 0 else 0.0
        
        # Saves per90
        saves_total = last5['saves'].sum() if 'saves' in last5.columns else 0
        
        return {
            'xgc_per90': to_per90(xgc_total),
            'clean_sheet_rate': cs_rate,
            'saves_per90': to_per90(saves_total),
        }
    
    def build_for_player(self, player_history: pd.DataFrame) -> dict:
        """Build features including defensive metrics."""
        # Get base features from parent
        features = super().build_for_player(player_history)
        
        # Add defensive features
        last5 = player_history.tail(5)
        defensive = self._compute_defensive_per90(last5)
        features.update(defensive)
        
        return features

print("DefensiveFeatureBuilder defined")

DefensiveFeatureBuilder defined


## 4. Build Feature Sets (Baseline vs Defensive)

In [5]:
# Baseline features (current production)
baseline_builder = FeatureBuilder()
baseline_df = baseline_builder.build_training_set(raw_df)
print(f"Baseline: {len(baseline_df):,} rows, {len(baseline_df.columns)} columns")

# Defensive features
defensive_builder = DefensiveFeatureBuilder()
defensive_df = defensive_builder.build_training_set(raw_df)
print(f"Defensive: {len(defensive_df):,} rows, {len(defensive_df.columns)} columns")

# Show new columns
new_cols = set(defensive_df.columns) - set(baseline_df.columns)
print(f"\nNew defensive columns: {new_cols}")

Baseline: 13,395 rows, 25 columns
Defensive: 13,395 rows, 28 columns

New defensive columns: {'clean_sheet_rate', 'saves_per90', 'xgc_per90'}


In [6]:
# Check defensive feature distributions
for col in ['xgc_per90', 'clean_sheet_rate', 'saves_per90']:
    if col in defensive_df.columns:
        print(f"{col}: mean={defensive_df[col].mean():.3f}, std={defensive_df[col].std():.3f}, "
              f"min={defensive_df[col].min():.3f}, max={defensive_df[col].max():.3f}")

xgc_per90: mean=0.914, std=2.023, min=0.000, max=84.600
clean_sheet_rate: mean=0.091, std=0.179, min=0.000, max=1.000
saves_per90: mean=0.000, std=0.000, min=0.000, max=0.000


## 5. Define Feature Sets for Ablation

In [7]:
# Current production features
BASELINE_FEATURES = BASE_FEATURES.copy()
print(f"Baseline features ({len(BASELINE_FEATURES)}): {BASELINE_FEATURES}")

# Defensive variant (excluding saves_per90 - no data available)
DEFENSIVE_FEATURES = BASELINE_FEATURES + ['xgc_per90', 'clean_sheet_rate']
print(f"\nDefensive features ({len(DEFENSIVE_FEATURES)}): {DEFENSIVE_FEATURES}")

Baseline features (16): ['per90_wmean', 'per90_wvar', 'mins_mean', 'appearances', 'is_home_next', 'games_since_first', 'goals_per90', 'assists_per90', 'bonus_per90', 'bps_per90', 'ict_per90', 'xg_per90', 'xa_per90', 'ict_per90_x_mins', 'xg_per90_x_apps', 'apps_x_goals']

Defensive features (18): ['per90_wmean', 'per90_wvar', 'mins_mean', 'appearances', 'is_home_next', 'games_since_first', 'goals_per90', 'assists_per90', 'bonus_per90', 'bps_per90', 'ict_per90', 'xg_per90', 'xa_per90', 'ict_per90_x_mins', 'xg_per90_x_apps', 'apps_x_goals', 'xgc_per90', 'clean_sheet_rate']


## 6. Train & Evaluate Models

In [8]:
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, mean_squared_error

def train_and_evaluate(df, feature_cols, name="Model"):
    """Train model and return predictions."""
    # Split by GW (temporal split)
    gws = sorted(df['gw'].unique())
    test_gws = gws[-4:]  # Last 4 GWs for test
    val_gws = gws[-8:-4]  # 4 GWs before that for val
    train_gws = gws[:-8]  # Rest for train
    
    train = df[df['gw'].isin(train_gws)]
    val = df[df['gw'].isin(val_gws)]
    test = df[df['gw'].isin(test_gws)]
    
    print(f"\n{name}:")
    print(f"  Train: GW {min(train_gws)}-{max(train_gws)} ({len(train):,} rows)")
    print(f"  Val:   GW {min(val_gws)}-{max(val_gws)} ({len(val):,} rows)")
    print(f"  Test:  GW {min(test_gws)}-{max(test_gws)} ({len(test):,} rows)")
    
    # Prepare data
    X_train = train[feature_cols].fillna(0)
    y_train = train['total_points']
    X_val = val[feature_cols].fillna(0)
    y_val = val['total_points']
    X_test = test[feature_cols].fillna(0)
    y_test = test['total_points']
    
    # Train LightGBM
    train_data = lgb.Dataset(X_train, label=y_train)
    val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)
    
    params = {
        'objective': 'regression',
        'metric': 'mae',
        'verbosity': -1,
        'num_leaves': 31,
        'learning_rate': 0.05,
        'feature_fraction': 0.8,
    }
    
    model = lgb.train(
        params,
        train_data,
        num_boost_round=200,
        valid_sets=[val_data],
        callbacks=[lgb.early_stopping(20), lgb.log_evaluation(0)],
    )
    
    # Evaluate
    pred_test = model.predict(X_test)
    mae = mean_absolute_error(y_test, pred_test)
    rmse = np.sqrt(mean_squared_error(y_test, pred_test))
    
    print(f"  Test MAE: {mae:.3f}, RMSE: {rmse:.3f}")
    
    # Return test predictions for regret analysis
    test_result = test.copy()
    test_result['predicted_points'] = pred_test
    
    return model, test_result, {'mae': mae, 'rmse': rmse}

In [9]:
# Train baseline model
baseline_model, baseline_preds, baseline_metrics = train_and_evaluate(
    baseline_df, BASELINE_FEATURES, "Baseline"
)

# Train defensive model
defensive_model, defensive_preds, defensive_metrics = train_and_evaluate(
    defensive_df, DEFENSIVE_FEATURES, "Defensive"
)


Baseline:
  Train: GW 6-15 (7,311 rows)
  Val:   GW 16-19 (3,020 rows)
  Test:  GW 20-23 (3,064 rows)
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[92]	valid_0's l1: 1.10175
  Test MAE: 1.068, RMSE: 1.967

Defensive:
  Train: GW 6-15 (7,311 rows)
  Val:   GW 16-19 (3,020 rows)
  Test:  GW 20-23 (3,064 rows)
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[79]	valid_0's l1: 1.10351
  Test MAE: 1.069, RMSE: 1.965


## 7. Captain Regret Comparison

In [10]:
def compute_captain_regret(predictions_df, squad_size=15):
    """Compute captain regret per GW.
    
    For each GW:
    1. Select top N players by predicted_points (simulated squad)
    2. Captain = argmax(predicted_points)
    3. Oracle = actual top scorer in squad
    4. Regret = (oracle_points - captain_points) * 2
    """
    results = []
    
    for gw, gw_df in predictions_df.groupby('gw'):
        # Simulate squad selection (top N by predicted)
        squad = gw_df.nlargest(squad_size, 'predicted_points')
        
        # Captain pick (argmax predicted)
        captain = squad.loc[squad['predicted_points'].idxmax()]
        captain_points = captain['total_points']
        
        # Oracle (best actual in squad)
        oracle = squad.loc[squad['total_points'].idxmax()]
        oracle_points = oracle['total_points']
        
        # Regret (doubled for captain)
        regret = (oracle_points - captain_points) * 2
        hit = 1 if captain['player_id'] == oracle['player_id'] else 0
        
        results.append({
            'gw': gw,
            'captain_name': captain.get('player_name', 'Unknown'),
            'captain_points': captain_points,
            'oracle_name': oracle.get('player_name', 'Unknown'),
            'oracle_points': oracle_points,
            'regret': regret,
            'hit': hit,
        })
    
    return pd.DataFrame(results)

# Compute regret for both models
baseline_regret = compute_captain_regret(baseline_preds)
defensive_regret = compute_captain_regret(defensive_preds)

print("Captain Regret Summary:")
print(f"  Baseline:  Mean={baseline_regret['regret'].mean():.2f}, Hit Rate={baseline_regret['hit'].mean()*100:.1f}%")
print(f"  Defensive: Mean={defensive_regret['regret'].mean():.2f}, Hit Rate={defensive_regret['hit'].mean()*100:.1f}%")

Captain Regret Summary:
  Baseline:  Mean=22.50, Hit Rate=0.0%
  Defensive: Mean=12.00, Hit Rate=0.0%


In [11]:
# Per-GW comparison
comparison = baseline_regret[['gw', 'captain_name', 'regret']].merge(
    defensive_regret[['gw', 'captain_name', 'regret']],
    on='gw',
    suffixes=('_baseline', '_defensive')
)
comparison['delta'] = comparison['regret_baseline'] - comparison['regret_defensive']
comparison['winner'] = comparison['delta'].apply(
    lambda x: 'Defensive' if x > 0 else ('Baseline' if x < 0 else 'Tie')
)

print("\nPer-GW Comparison:")
print(comparison.to_string(index=False))

print(f"\nDefensive wins: {(comparison['winner'] == 'Defensive').sum()}")
print(f"Baseline wins: {(comparison['winner'] == 'Baseline').sum()}")
print(f"Ties: {(comparison['winner'] == 'Tie').sum()}")


Per-GW Comparison:
 gw captain_name_baseline  regret_baseline captain_name_defensive  regret_defensive  delta    winner
 20                 Foden               14                  Foden                14      0       Tie
 21                Gordon               26                  Foden                14     12 Defensive
 22                 Bijol               30               E.Le Fée                 4     26 Defensive
 23             Tavernier               20                 Thiago                16      4 Defensive

Defensive wins: 3
Baseline wins: 0
Ties: 1


## 7b. Transfer Regret Comparison

In [18]:
def compute_transfer_regret(predictions_df, top_k=5):
    """Compute transfer regret per GW.
    
    For each GW:
    1. Rank all players by predicted_points
    2. Compare model's top-K picks vs oracle's top-K actual scorers
    3. Regret = sum(oracle_top_k_points) - sum(model_top_k_points)
    """
    results = []
    
    for gw, gw_df in predictions_df.groupby('gw'):
        # Model's top picks
        model_picks = gw_df.nlargest(top_k, 'predicted_points')
        model_points = model_picks['total_points'].sum()
        
        # Oracle's best picks (hindsight)
        oracle_picks = gw_df.nlargest(top_k, 'total_points')
        oracle_points = oracle_picks['total_points'].sum()
        
        # Regret
        regret = oracle_points - model_points
        
        # Top pick comparison
        model_top = model_picks.iloc[0]
        oracle_top = oracle_picks.iloc[0]
        
        results.append({
            'gw': gw,
            'model_top': model_top.get('player_name', 'Unknown'),
            'model_top_pts': model_top['total_points'],
            'oracle_top': oracle_top.get('player_name', 'Unknown'),
            'oracle_top_pts': oracle_top['total_points'],
            'model_total': model_points,
            'oracle_total': oracle_points,
            'regret': regret,
        })
    
    return pd.DataFrame(results)

# Compute transfer regret for both models
baseline_transfer = compute_transfer_regret(baseline_preds)
defensive_transfer = compute_transfer_regret(defensive_preds)

print("Transfer Regret Summary (Top-5 picks):")
print(f"  Baseline:  Mean={baseline_transfer['regret'].mean():.2f} pts/GW")
print(f"  Defensive: Mean={defensive_transfer['regret'].mean():.2f} pts/GW")
print(f"  Delta: {baseline_transfer['regret'].mean() - defensive_transfer['regret'].mean():+.2f} pts/GW")

Transfer Regret Summary (Top-5 picks):
  Baseline:  Mean=43.00 pts/GW
  Defensive: Mean=46.25 pts/GW
  Delta: -3.25 pts/GW


In [19]:
# Per-GW transfer comparison
transfer_comp = baseline_transfer[['gw', 'model_top', 'regret']].merge(
    defensive_transfer[['gw', 'model_top', 'regret']],
    on='gw',
    suffixes=('_baseline', '_defensive')
)
transfer_comp['delta'] = transfer_comp['regret_baseline'] - transfer_comp['regret_defensive']
transfer_comp['winner'] = transfer_comp['delta'].apply(
    lambda x: 'Defensive' if x > 0 else ('Baseline' if x < 0 else 'Tie')
)

print("\nPer-GW Transfer Comparison:")
print(transfer_comp.to_string(index=False))

print(f"\nDefensive wins: {(transfer_comp['winner'] == 'Defensive').sum()}")
print(f"Baseline wins: {(transfer_comp['winner'] == 'Baseline').sum()}")
print(f"Ties: {(transfer_comp['winner'] == 'Tie').sum()}")


Per-GW Transfer Comparison:
 gw model_top_baseline  regret_baseline model_top_defensive  regret_defensive  delta    winner
 20              Foden               59               Foden                56      3 Defensive
 21             Gordon               50               Foden                45      5 Defensive
 22              Bijol               20            E.Le Fée                40    -20  Baseline
 23          Tavernier               43              Thiago                44     -1  Baseline

Defensive wins: 2
Baseline wins: 2
Ties: 0


## 8. Feature Importance

In [13]:
# Defensive model feature importance
importance = pd.DataFrame({
    'feature': DEFENSIVE_FEATURES,
    'importance': defensive_model.feature_importance('gain')
}).sort_values('importance', ascending=False)

print("Feature Importance (Defensive Model):")
print(importance.to_string(index=False))

# Highlight defensive features
print("\nDefensive feature importance:")
for feat in ['xgc_per90', 'clean_sheet_rate']:
    if feat in importance['feature'].values:
        imp = importance[importance['feature'] == feat]['importance'].values[0]
        rank = list(importance['feature']).index(feat) + 1
        print(f"  {feat}: {imp:.1f} (rank {rank}/{len(DEFENSIVE_FEATURES)})")

Feature Importance (Defensive Model):
          feature   importance
 ict_per90_x_mins 82385.368123
        mins_mean 54315.099892
        xgc_per90 15083.904095
      per90_wmean 11045.807807
        ict_per90  8988.462816
         xg_per90  8882.541615
      appearances  8517.051239
       per90_wvar  8266.163387
        bps_per90  8180.186914
         xa_per90  7947.145102
    assists_per90  6681.496105
  xg_per90_x_apps  6247.755585
      bonus_per90  5065.094896
 clean_sheet_rate  4094.677103
games_since_first  3994.276903
      goals_per90  3123.432407
     is_home_next  2516.732714
     apps_x_goals  1351.927213

Defensive feature importance:
  xgc_per90: 15083.9 (rank 3/18)
  clean_sheet_rate: 4094.7 (rank 14/18)


## 9. Position-Specific Analysis

In [16]:
# Compare MAE by position
def mae_by_position(preds_df, name):
    results = []
    if 'position' not in preds_df.columns:
        print(f"Warning: 'position' column not in {name}")
        return pd.DataFrame()
    for pos in ['GKP', 'DEF', 'MID', 'FWD']:
        pos_df = preds_df[preds_df['position'] == pos]
        if len(pos_df) > 0:
            mae = mean_absolute_error(pos_df['total_points'], pos_df['predicted_points'])
            results.append({'position': pos, 'mae': mae, 'n': len(pos_df)})
    return pd.DataFrame(results)

# Check if position column exists in both
print(f"Baseline columns: {list(baseline_preds.columns)[:8]}...")
print(f"Defensive columns: {list(defensive_preds.columns)[:8]}...")

baseline_pos = mae_by_position(baseline_preds, "baseline")
defensive_pos = mae_by_position(defensive_preds, "defensive")

if len(baseline_pos) > 0 and len(defensive_pos) > 0:
    pos_comparison = baseline_pos.merge(defensive_pos, on='position', suffixes=('_baseline', '_defensive'))
    pos_comparison['delta_mae'] = pos_comparison['mae_baseline'] - pos_comparison['mae_defensive']

    print("\nMAE by Position:")
    print(pos_comparison.to_string(index=False))
    print("\n(Positive delta = defensive is better)")
else:
    # Just show overall MAE comparison
    print("\nSkipping per-position analysis")
    print("Overall MAE comparison already shown above")

Baseline columns: ['player_id', 'player_name', 'team_name', 'team_id', 'position', 'gw', 'total_points', 'minutes']...
Defensive columns: ['player_id', 'player_name', 'team_name', 'team_id', 'position', 'gw', 'total_points', 'minutes']...

Skipping per-position analysis
Overall MAE comparison already shown above


## 10. Verdict

In [17]:
# Summary
baseline_mean_regret = baseline_regret['regret'].mean()
defensive_mean_regret = defensive_regret['regret'].mean()
delta = baseline_mean_regret - defensive_mean_regret

print("="*70)
print("ABLATION VERDICT: Defensive Features")
print("="*70)
print(f"\nBaseline Mean Regret:  {baseline_mean_regret:.2f} pts/GW")
print(f"Defensive Mean Regret: {defensive_mean_regret:.2f} pts/GW")
print(f"Delta: {delta:+.2f} pts/GW")

if delta > 1.0:
    print(f"\n✅ ACCEPT: Defensive features reduce regret by {delta:.2f} pts/GW")
    print("   Recommendation: Add to production")
elif delta < -1.0:
    print(f"\n❌ REJECT: Defensive features increase regret by {-delta:.2f} pts/GW")
    print("   Recommendation: Do not add to production")
else:
    print(f"\n⚠️  INCONCLUSIVE: Delta ({delta:.2f}) within noise threshold (±1.0)")
    print("   Recommendation: Need more data or position-specific analysis")

ABLATION VERDICT: Defensive Features

Baseline Mean Regret:  22.50 pts/GW
Defensive Mean Regret: 12.00 pts/GW
Delta: +10.50 pts/GW

✅ ACCEPT: Defensive features reduce regret by 10.50 pts/GW
   Recommendation: Add to production


## 11. Full Walk-Forward Backtest (GW 6-23)

In [20]:
def walk_forward_backtest(df, feature_cols, name="Model", min_train_gws=5):
    """Walk-forward validation: train on GW 1..t, predict t+1.
    
    Returns predictions for all GWs from min_train_gws+1 onwards.
    """
    gws = sorted(df['gw'].unique())
    all_preds = []
    
    for i, test_gw in enumerate(gws[min_train_gws:], start=min_train_gws):
        train_gws = gws[:i]
        train = df[df['gw'].isin(train_gws)]
        test = df[df['gw'] == test_gw]
        
        if len(test) == 0:
            continue
            
        # Train model
        X_train = train[feature_cols].fillna(0)
        y_train = train['total_points']
        X_test = test[feature_cols].fillna(0)
        
        train_data = lgb.Dataset(X_train, label=y_train)
        params = {
            'objective': 'regression',
            'metric': 'mae',
            'verbosity': -1,
            'num_leaves': 31,
            'learning_rate': 0.05,
            'feature_fraction': 0.8,
        }
        model = lgb.train(params, train_data, num_boost_round=100)
        
        # Predict
        test_result = test.copy()
        test_result['predicted_points'] = model.predict(X_test)
        all_preds.append(test_result)
    
    print(f"{name}: Walk-forward on GW {gws[min_train_gws]}-{gws[-1]}")
    return pd.concat(all_preds, ignore_index=True)

# Run walk-forward for both models
print("Running walk-forward backtest (this may take a minute)...")
baseline_wf = walk_forward_backtest(baseline_df, BASELINE_FEATURES, "Baseline")
defensive_wf = walk_forward_backtest(defensive_df, DEFENSIVE_FEATURES, "Defensive")

Running walk-forward backtest (this may take a minute)...
Baseline: Walk-forward on GW 11-23
Defensive: Walk-forward on GW 11-23


In [21]:
# Captain regret on full backtest
baseline_wf_regret = compute_captain_regret(baseline_wf)
defensive_wf_regret = compute_captain_regret(defensive_wf)

print("\n" + "="*70)
print("FULL WALK-FORWARD CAPTAIN REGRET")
print("="*70)
print(f"Baseline:  Mean={baseline_wf_regret['regret'].mean():.2f} pts/GW, Hit={baseline_wf_regret['hit'].mean()*100:.1f}%")
print(f"Defensive: Mean={defensive_wf_regret['regret'].mean():.2f} pts/GW, Hit={defensive_wf_regret['hit'].mean()*100:.1f}%")
print(f"Delta: {baseline_wf_regret['regret'].mean() - defensive_wf_regret['regret'].mean():+.2f} pts/GW")

# Transfer regret on full backtest
baseline_wf_transfer = compute_transfer_regret(baseline_wf)
defensive_wf_transfer = compute_transfer_regret(defensive_wf)

print("\n" + "="*70)
print("FULL WALK-FORWARD TRANSFER REGRET")
print("="*70)
print(f"Baseline:  Mean={baseline_wf_transfer['regret'].mean():.2f} pts/GW")
print(f"Defensive: Mean={defensive_wf_transfer['regret'].mean():.2f} pts/GW")
print(f"Delta: {baseline_wf_transfer['regret'].mean() - defensive_wf_transfer['regret'].mean():+.2f} pts/GW")


FULL WALK-FORWARD CAPTAIN REGRET
Baseline:  Mean=13.54 pts/GW, Hit=7.7%
Defensive: Mean=14.46 pts/GW, Hit=30.8%
Delta: -0.92 pts/GW

FULL WALK-FORWARD TRANSFER REGRET
Baseline:  Mean=51.46 pts/GW
Defensive: Mean=54.23 pts/GW
Delta: -2.77 pts/GW


## 12. Position-Conditional Features (xGC only for DEF/GKP)

In [ ]:
# Position-conditional: Zero out defensive features for MID/FWD
# Position mapping: 1=GKP, 2=DEF, 3=MID, 4=FWD
POS_MAP = {1: 'GKP', 2: 'DEF', 3: 'MID', 4: 'FWD'}

def make_position_conditional(df):
    """Zero out defensive features for non-defenders (MID/FWD = position 3,4)."""
    df = df.copy()
    for col in ['xgc_per90', 'clean_sheet_rate']:
        if col in df.columns:
            # Zero for MID (3) and FWD (4)
            mask = df['position'].isin([3, 4])
            df.loc[mask, col] = 0.0
    return df

conditional_df = make_position_conditional(defensive_df)
print("Position-conditional features applied")
print(f"Positions in data: {[POS_MAP.get(p, p) for p in sorted(conditional_df['position'].unique())]}")

# Show stats by position type
def_gkp = conditional_df[conditional_df['position'].isin([1, 2])]
mid_fwd = conditional_df[conditional_df['position'].isin([3, 4])]
print(f"DEF/GKP count: {len(def_gkp)}, xgc_per90 mean: {def_gkp['xgc_per90'].mean():.3f}")
print(f"MID/FWD count: {len(mid_fwd)}, xgc_per90 mean: {mid_fwd['xgc_per90'].mean():.3f}")

Position-conditional features applied
Positions in data: [1 2 3 4]
DEF/GKP count: 0
MID/FWD count: 0

DEF/GKP xgc_per90 mean: nan
MID/FWD xgc_per90 mean: nan


In [ ]:
# Walk-forward with position-conditional features
print("Running position-conditional walk-forward...")
conditional_wf = walk_forward_backtest(conditional_df, DEFENSIVE_FEATURES, "Conditional")

# Captain regret
conditional_wf_regret = compute_captain_regret(conditional_wf)

# Transfer regret
conditional_wf_transfer = compute_transfer_regret(conditional_wf)

print("\n" + "="*70)
print("POSITION-CONDITIONAL RESULTS")
print("="*70)
print(f"\nCaptain Regret:")
print(f"  Baseline:    {baseline_wf_regret['regret'].mean():.2f} pts/GW")
print(f"  Defensive:   {defensive_wf_regret['regret'].mean():.2f} pts/GW")
print(f"  Conditional: {conditional_wf_regret['regret'].mean():.2f} pts/GW")

print(f"\nTransfer Regret:")
print(f"  Baseline:    {baseline_wf_transfer['regret'].mean():.2f} pts/GW")
print(f"  Defensive:   {defensive_wf_transfer['regret'].mean():.2f} pts/GW")
print(f"  Conditional: {conditional_wf_transfer['regret'].mean():.2f} pts/GW")

## 13. Final Comparison Table

In [ ]:
# Final summary table
summary = pd.DataFrame({
    'Model': ['Baseline (16 feat)', 'Defensive (18 feat)', 'Conditional (18 feat)'],
    'Captain Regret': [
        baseline_wf_regret['regret'].mean(),
        defensive_wf_regret['regret'].mean(),
        conditional_wf_regret['regret'].mean(),
    ],
    'Captain Hit%': [
        baseline_wf_regret['hit'].mean() * 100,
        defensive_wf_regret['hit'].mean() * 100,
        conditional_wf_regret['hit'].mean() * 100,
    ],
    'Transfer Regret': [
        baseline_wf_transfer['regret'].mean(),
        defensive_wf_transfer['regret'].mean(),
        conditional_wf_transfer['regret'].mean(),
    ],
})

# Highlight winners
summary['Captain Winner'] = summary['Captain Regret'] == summary['Captain Regret'].min()
summary['Transfer Winner'] = summary['Transfer Regret'] == summary['Transfer Regret'].min()

print("="*80)
print("ABLATION SUMMARY: DEFENSIVE FEATURES")
print("="*80)
print(f"\nGWs tested: {baseline_wf['gw'].min()}-{baseline_wf['gw'].max()} (n={baseline_wf['gw'].nunique()})")
print("\n" + summary.to_string(index=False))

# Verdict
best_captain = summary.loc[summary['Captain Regret'].idxmin(), 'Model']
best_transfer = summary.loc[summary['Transfer Regret'].idxmin(), 'Model']

print(f"\n\nVERDICT:")
print(f"  Best for Captain:  {best_captain}")
print(f"  Best for Transfer: {best_transfer}")